In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Get player links

In [32]:
from IPython.display import clear_output
base_url = 'https://www.nbadraft.net/players/'

player_links = []

for page_num in range(1, 366):
    if page_num == 1:
        page_url = base_url
    else:
        page_url = f'{base_url}page/{page_num}/'
    
    # Send a GET request to the page URL
    response = requests.get(page_url)
    
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        # parent div containing all player links
        parent_div = soup.find('div', class_='wf-container')

        # Find all the player links on the page
        links = parent_div.find_all('a')
        
        # extract href attribute and append to player_links
        n = 0
        for link in links:
            link = link['href']
            if base_url in link:
                player_links.append(link)
                n += 1
        clear_output(wait=True)
        print(f'Found {n} player links in {page_url}')

Found 4 player links in https://www.nbadraft.net/players/page/365/


Save as a txt file because that took a while and we don't want to do it again

In [1]:
# with open('../data/nbadraft_player_links.txt', 'w') as file:
#     for link in player_links:
#         file.write(link + "\n")

Scrape player links for strengths/weaknesses

In [3]:
import numpy as np
f_links = '../data/nbadraft_player_links.txt'

with open(f_links, 'r') as file:
    player_links = file.readlines()

player_links = [link.strip() for link in player_links]

In [112]:
# for some reason we have duplciates - only get unique
import numpy as np
player_links = list(np.unique(player_links))

In [179]:
ind = player_links.index(player_link)
player_links[ind:]

['https://www.nbadraft.net/players/cleanthony-early/',
 'https://www.nbadraft.net/players/cleveland-melvin/',
 'https://www.nbadraft.net/players/clevon-brown/',
 'https://www.nbadraft.net/players/cliff-alexander/',
 'https://www.nbadraft.net/players/clifford-omoruyi/',
 'https://www.nbadraft.net/players/clifton-moore/',
 'https://www.nbadraft.net/players/clint-capela/',
 'https://www.nbadraft.net/players/coby-karl/',
 'https://www.nbadraft.net/players/coby-white/',
 'https://www.nbadraft.net/players/codi-miller-mcintyre/',
 'https://www.nbadraft.net/players/cody-doolin/',
 'https://www.nbadraft.net/players/cody-larson/',
 'https://www.nbadraft.net/players/cody-martin/',
 'https://www.nbadraft.net/players/cody-riley/',
 'https://www.nbadraft.net/players/cody-williams/',
 'https://www.nbadraft.net/players/cody-zeller/',
 'https://www.nbadraft.net/players/coen-carr/',
 'https://www.nbadraft.net/players/colbey-ross/',
 'https://www.nbadraft.net/players/colby-jones/',
 'https://www.nbadraft

In [ ]:
data = []
missing_players = []

In [213]:
from IPython.display import clear_output

data = []
missing_players = []

# ind = player_links.index(player_link)
ind = 0

for player_link in player_links[ind:]:
    # for testing
    # player_link = 'https://www.nbadraft.net/players/james-harden/'
    # player_link = 'https://www.nbadraft.net/players/amen-thompson/'
    # player_link = 'https://www.nbadraft.net/players/acie-law'

    clear_output(wait=True)
    player_name = player_link.rsplit('/',2)[-2]
    response = requests.get(player_link)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        for identify in ['b', 'strong']:

            # GET STRENGTHS TEXT
            # loop for all potential strings
            for text in ['Strengths:', 'Strengths: ']:
                strength_obj = soup.find(identify, text=text) # first is most recent so we use that
                if strength_obj is not None:
                    break
            if strength_obj is None:
                continue

            # GET WEAKNESSES TEXT
            # loop through all potential strings
            for text in ['Weaknesses:', 'Weakness:', 'Weaknesses: ']:
                weak_obj = soup.find(identify, text=text)
                if weak_obj is not None:
                    break

            if (strength_obj is not None) & (weak_obj is not None):
                break

        if (weak_obj is None) | (strength_obj is None):
            print(f"Failed to scrape data for {player_name}. No weaknesses available")
            continue
        try:
            strength_text = strength_obj.find_next_sibling(text=True).strip()
            weak_text = weak_obj.find_next_sibling(text=True).strip()
        except AttributeError:
            try:
                strength_text = strength_obj.find_next_sibling().text.strip()
                weak_text = strength_obj.find_next_sibling().text.strip()
            except AttributeError:
                continue
        try:
            year = int(soup.find('div',class_='mock-year').find('span', class_='label').text[:4])
        # missing year is allowed
        except AttributeError:
            year = np.nan
        try:
            overall = int(soup.find('div',class_='overall').find('span', class_='value').text)
        # missing overall attribute not allowed
        except AttributeError:
            continue
        player_attr_obj = soup.find('div', class_='player-attributes')

        # attribute scores
        attr_values = player_attr_obj.find_all('div', class_='div-table-cell attribute-value')
        attr_names = player_attr_obj.find_all('div', class_='div-table-cell attribute-name')
        attr_dict = {}
        try:
            for name, value in zip(attr_names, attr_values):
                attr_dict[name.text.replace(' ', '')] = int(value.text)
        # don't allow missing attribute values
        except ValueError:
            continue
            
        # Append the data to the list
        tmp_dict = {
            "player": player_name,
            "strengths": strength_text,
            "weaknesses": weak_text,
            'overall': overall
        }
        tmp_dict.update(attr_dict)
        data.append(tmp_dict)
        print(f'Success! Scraped strengths/weaknesses for {player_name}')
        # break
    else:
        missing_players.append(player_name)
        print(f"Failed to scrape data for {player_name}. Status code: {response.status_code}")
        # break

    # break

# Create a pandas dataframe from the scraped data
df = pd.DataFrame(data)

Failed to scrape data for zylan-cheatham. No weaknesses available


In [216]:
df.to_csv('../data/nbadraft_strengths_weaknesses.csv')

In [39]:
df

,player,draft_year,strengths,weaknesses
0,aaron-bradshaw,2024,7’0 center … Good size and length … Athletic w...,Has much room to get stronger in both upper an...
1,aaron-gordon,2014,A freakish athlete with incredible explosivene...,Gordon’s main weaknesses revolve around his la...
2,aaron-gray,Over,Unlike many post players Gray doesnt put the b...,Athletically Gray is not exceptionally quick o...
3,aaron-harrison,Big,Strengths: Leading scorer of a loaded 2015 Ken...,Weaknesses: Really struggled with consistency ...
4,aaron-henry,Big,Entering Tom Izzo’s storied program as a three...,"Despite his high scoring average, Henry was in..."
...,...,...,...,...
732,zhaire-smith,2018,Has an underrated feel for the game … High bas...,"As promising as he is, he’s still a project of..."
733,zhang-songtao,Over,Very coordinated big man with solid footwork …...,Release on his shot is rather slow … Jumps pre...
734,zhou-qi,2016,"Center prospect with great size, length and mo...",Lacks overall strength and weight to be effect...
735,ziaire-williams,2021,High level athlete who uses his size and verti...,Williams struggled to stay on the court for th...
